In [17]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import time
import csv
import configparser
from Auction import Auction

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
navigator = webdriver.Chrome(ChromeDriverManager().install())
navigator.get(config['URL']['NORDESTE'])

/tmp/ipykernel_14038/3342353510.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navigator = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
auction = Auction(navigator, config)

##### Get Title

In [5]:
auction_title = auction.get_element_text('AUCTION-TITLE').replace('/','-')
auction_title

'LEILÃO DE VEÍCULOS NORDESTE - 08-03-23'

In [6]:
time.sleep(10)
total_results = int(auction.get_element_text('TOTAL-RESULTS').split(' ')[0])
total_results

121

#### Click on car

In [7]:
auction.click_first_car()

In [8]:
all_vehicles = auction.get_all_vehicles(total_results=total_results)

In [9]:
all_vehicles

[{'BATCH': '1',
  'VEHICLE': 'renault/kwid zen 1.0 mecanico',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '23.368',
  'YEAR': '20/21',
  'FUEL': 'Flex',
  'COLOR': 'BRANCA',
  'FIPE': 'R$ 49.991,00',
  'YARD': 'PATIO 2 - FORTALEZA/CE',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'BATCH': '2',
  'VEHICLE': 'chevrolet/prisma joye 1.0 mecanico (4p)',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '58.003',
  'YEAR': '19/19',
  'FUEL': 'Flex',
  'COLOR': 'BRANCA',
  'FIPE': 'R$ 53.894,00',
  'YARD': 'PATIO 2 - FORTALEZA/CE',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'BATCH': '3',
  'VEHICLE': 'fiat/strada hard working cabine simples cc e',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '102.032',
  'YEAR': '19/19',
  'FUEL': 'Flex',
  'COLOR': 'PRATA',
  'FIPE': 'R$ 61.111,00',
  'YARD': 'PATIO 2 - FORTALEZA/CE',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'

In [18]:
auction.write_csv(name = auction_title, cars = all_vehicles)

In [19]:
vehicles = pd.read_csv('LEILÃO DE VEÍCULOS NORDESTE - 08-03-23.csv')

In [20]:
vehicles.head(2)

,BATCH,VEHICLE,BANK,KM,YEAR,FUEL,COLOR,FIPE,YARD,RESUME-TYPE,TYPE,HAS-KEY
0,1,renault/kwid zen 1.0 mecanico,BANCO SANTANDER S/A,23.368,20/21,Flex,BRANCA,"R$ 49.991,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
1,2,chevrolet/prisma joye 1.0 mecanico (4p),BANCO SANTANDER S/A,58.003,19/19,Flex,BRANCA,"R$ 53.894,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim


In [21]:
vehicles.sort_values(by='KM').head(10)

,BATCH,VEHICLE,BANK,KM,YEAR,FUEL,COLOR,FIPE,YARD,RESUME-TYPE,TYPE,HAS-KEY
78,139,honda/pcx 150 sport abs,BANCO SANTANDER S/A,8.414,20/21,Gasolina,PRATA,"R$ 17.629,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
63,120,honda/cg 160 fan,BANCO SANTANDER S/A,11.884,22/22,Flex,VERMELHA,"R$ 15.772,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
68,125,haojian/avelloz az1,BANCO SANTANDER S/A,11.922,21/22,Gasolina,BRANCA,"R$ 7.499,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
77,137,kawasak/z400,BANCO SANTANDER S/A,16.192,21/21,Gasolina,PRETA,"R$ 28.924,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
45,84,honda/cg 125 cargo ks,NORSA REFRIGERANTES S/A,17.916,14/14,Gasolina,BRANCA,"R$ 8.715,00",PATIO FORTALEZA/CE,Frota,Motos,Não
0,1,renault/kwid zen 1.0 mecanico,BANCO SANTANDER S/A,23.368,20/21,Flex,BRANCA,"R$ 49.991,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
15,19,honda/cg 125 cargo ks,NORSA REFRIGERANTES S/A,23.536,14/14,Gasolina,BRANCA,"R$ 8.715,00",PATIO FORTALEZA/CE,Frota,Motos,Não
30,34,volkswagen/gol mc4 (4p),BANCO VOLKSWAGEN S/A,30.498,19/20,Flex,VERMELHA,"R$ 51.823,00",PATIO FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
24,28,volkswagen/polo comfortline ad,BANCO VOLKSWAGEN S/A,50.641,19/20,Flex,AZUL,"R$ 84.521,00",PATIO FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
108,213,ford/fiesta flex (4p),BANCO SANTANDER S/A,51.049,13/14,Flex,BRANCA,"R$ 30.046,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
